In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.6.0+cu124


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset, Subset
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, log_loss
from munkres import Munkres

In [14]:
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [15]:
class PairDataset(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = pairs
        self.labels = labels

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx][0], self.pairs[idx][1], self.labels[idx]

In [16]:
class SiameseNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=256):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)

    def forward_once(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        dist = F.pairwise_distance(out1, out2)
        return dist, out1, out2

In [17]:
class OrthoLinear(nn.Module):
    def __init__(self, in_dim, out_dim, eps=1e-4):
        super().__init__()
        self.fc = nn.Linear(in_dim, out_dim)
        self.eps = eps

    def forward(self, x):
        Y_tilde = self.fc(x)
        gram = Y_tilde.T @ Y_tilde + self.eps * torch.eye(Y_tilde.shape[1], device=x.device)
        L = torch.linalg.cholesky(gram)
        L_inv = torch.inverse(L)
        return Y_tilde @ L_inv.T

In [ ]:
class SpectralNet(nn.Module):
    def __init__(self, input_dim, n_clusters, hidden_dim=256):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.ortho = OrthoLinear(hidden_dim, n_clusters)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.ortho(x)

In [18]:
def contrastive_loss(distance, label, margin=1.0):
    pos = label * torch.pow(distance, 2)
    neg = (1 - label) * torch.pow(torch.clamp(margin - distance, min=0.0), 2)
    return torch.mean(pos + neg)

def spectral_loss(Y, W):
    D = torch.diag(W.sum(axis=1))
    L = D - W
    num = torch.trace(Y.T @ L @ Y)
    denom = torch.trace(Y.T @ D @ Y)
    return num / (denom + 1e-12)

In [19]:
def compute_affinity(X, scale, n_neighbors=20):
    nbrs = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)
    distances, indices = distances[:, 1:], indices[:, 1:]
    W = np.zeros((len(X), len(X)))
    for i in range(len(X)):
        for j in range(n_neighbors):
            W[i, indices[i, j]] = np.exp(-distances[i, j] ** 2 / (2 * scale ** 2))
            W[indices[i, j], i] = W[i, indices[i, j]]
    return W

def compute_scale(X, n_neighbors=20):
    nbrs = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
    distances, _ = nbrs.kneighbors(X)
    return np.median(distances[:, -1])

def calculate_accuracy(y_pred, y_true, n_clusters):
    cm = confusion_matrix(y_true, y_pred)
    cost = np.zeros((n_clusters, n_clusters))
    for i in range(n_clusters):
        for j in range(n_clusters):
            cost[i, j] = cm[:, j].sum() - cm[i, j]
    m = Munkres()
    mapping = m.compute(cost.tolist())
    new_labels = np.zeros_like(y_pred)
    for row, col in mapping:
        new_labels[y_pred == row] = col
    return (new_labels == y_true).mean()

In [20]:
def train_siamese(siamese_net, dataloader, epochs=50, lr=1e-3, device='cpu'):
    siamese_net.to(device)
    opt = optim.Adam(siamese_net.parameters(), lr=lr)
    for ep in range(epochs):
        siamese_net.train()
        total_loss = 0.0
        for x1, x2, labels in dataloader:
            x1 = x1.to(device).float()
            x2 = x2.to(device).float()
            labels = labels.to(device).float()
            dist, _, _ = siamese_net(x1, x2)
            loss = contrastive_loss(dist, labels)
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item() * x1.size(0)
        avg_loss = total_loss / len(dataloader.dataset)
        print(f"[Siamese] Epoch {ep + 1}/{epochs}, Avg Loss={avg_loss:.6f}")
    siamese_net.to('cpu')
    return siamese_net

def train_spectral(spectral_net, X_train, W, epochs=50, lr=1e-3, tol=1e-6, device='cpu'):
    spectral_net.to(device)
    opt = optim.Adam([
        {'params': spectral_net.fc1.parameters()},
        {'params': spectral_net.fc2.parameters()},
        {'params': spectral_net.ortho.fc.parameters()}
    ], lr=lr)

    X_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
    W_tensor = torch.tensor(W, dtype=torch.float32, device=device)

    prev_loss = float('inf')
    for ep in range(epochs):
        spectral_net.train()
        Y = spectral_net(X_tensor)  # (n_samples, n_clusters)
        loss = spectral_loss(Y, W_tensor)
        opt.zero_grad()
        loss.backward()
        opt.step()

        loss_item = loss.item()
        print(f"[SpectralNet] Epoch {ep + 1}/{epochs}, Loss={loss_item:.8f}")

        if abs(prev_loss - loss_item) < tol:
            print("SpectralNet converged (tol reached).")
            break
        prev_loss = loss_item
    spectral_net.to('cpu')
    return spectral_net

In [23]:
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # ====== Load PneumoniaMNIST ======
    data = np.load('/home/snu/Downloads/pneumoniamnist_224.npz', allow_pickle=True)
    all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
    all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

    images = all_images.astype(np.float32) / 255.0
    images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
    labels = all_labels.astype(np.int64)

    # ====== Select 2000 samples per class ======
    selected_indices = []
    num_per_class = 2000
    classes = np.unique(labels)
    for c in classes:
        class_idx = np.where(labels == c)[0]
        chosen = np.random.choice(class_idx, size=min(num_per_class, len(class_idx)), replace=False)
        selected_indices.extend(chosen)

    selected_indices = np.array(selected_indices)
    images = images[selected_indices]
    labels = labels[selected_indices]

    # ====== Create dataset ======
    dataset = TensorDataset(torch.tensor(images), torch.tensor(labels))
    loader = DataLoader(dataset, batch_size=64, shuffle=False)

    # ====== Load ViT-DINO ======
    vit = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
    vit.eval().to(device)

    feats, y_list = [], []
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs = imgs.to(device).float()
            f = vit(imgs)
            if isinstance(f, (list, tuple)):
                f = f[0]
            feats.append(f.cpu())
            y_list.append(lbls)

    X = torch.cat(feats, dim=0).numpy().astype(np.float32)
    y = torch.cat(y_list, dim=0).numpy().astype(np.int64)

    # Shuffle
    perm = np.random.permutation(len(X))
    X, y = X[perm], y[perm]
    print("Balanced subset:", X.shape, y.shape)

    num_nodes, num_feats = X.shape
    n_clusters = 2
    hidden_dim = 256
    batch_size = 16
    n_neighbors = 20

    # ====== Build siamese pairs ======
    pairs, labels_pairs = [], []
    nbrs = NearestNeighbors(n_neighbors=n_neighbors + 1).fit(X)
    distances, indices = nbrs.kneighbors(X)
    for i in range(len(X)):
        for j in indices[i, 1:]:
            pairs.append([X[i], X[j]])
            labels_pairs.append(1)
        all_indices = set(range(len(X)))
        neighbor_set = set(indices[i, 1:])
        non_neighbors = list(all_indices - neighbor_set - {i})
        j = np.random.choice(non_neighbors)
        pairs.append([X[i], X[j]])
        labels_pairs.append(0)

    dataset_pairs = PairDataset(pairs, labels_pairs)
    dataloader_pairs = DataLoader(dataset_pairs, batch_size=batch_size, shuffle=True)

    # ====== Train Siamese ======
    siamese = SiameseNet(num_feats, hidden_dim)
    siamese = train_siamese(siamese, dataloader_pairs, epochs=50, device=device)

    with torch.no_grad():
        X_embed = siamese.forward_once(torch.tensor(X, dtype=torch.float32)).numpy()

    # ====== Train SpectralNet ======
    scale = compute_scale(X_embed, n_neighbors=n_neighbors)
    W = compute_affinity(X_embed, scale, n_neighbors=n_neighbors)
    spectral = SpectralNet(num_feats, n_clusters, hidden_dim)
    spectral = train_spectral(spectral, X, W, epochs=50, device=device)

    with torch.no_grad():
        Y = spectral(torch.tensor(X, dtype=torch.float32)).numpy()
        y_pred_proba = F.softmax(torch.tensor(Y), dim=1).numpy()

    kmeans = KMeans(n_clusters=n_clusters, n_init=20)
    y_pred = kmeans.fit_predict(Y)

    acc_score = calculate_accuracy(y_pred, y, n_clusters)
    acc_score_inverted = calculate_accuracy(1 - y_pred, y, n_clusters)
    if acc_score_inverted > acc_score:
        acc_score = acc_score_inverted
        y_pred = 1 - y_pred

    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    ll = log_loss(y, y_pred_proba)

    print("Final clustering accuracy:", acc_score)
    print("Precision:", prec)
    print("Recall:", rec)
    print("F1 Score:", f1)
    print("Log Loss:", ll)

    return {"accuracy": acc_score, "precision": prec, "recall": rec, "f1": f1, "log_loss": ll}

# --------------------------
# Multi-runs
# --------------------------
if __name__ == "__main__":
    num_runs = 10
    all_results = {"accuracy": [], "precision": [], "recall": [], "f1": [], "log_loss": []}
    for run in range(num_runs):
        print(f"\n--- Run {run+1}/{num_runs} ---")
        set_seed(run)
        res = main()
        for k in all_results.keys():
            all_results[k].append(res[k])

    print("\n=== FINAL SUMMARY ===")
    for metric, vals in all_results.items():
        print(f"{metric:>10} | mean={np.mean(vals):.4f} ± {np.std(vals):.4f}")



--- Run 1/10 ---


Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Balanced subset: (3583, 768) (3583,)
[Siamese] Epoch 1/50, Avg Loss=0.034467
[Siamese] Epoch 2/50, Avg Loss=0.032635
[Siamese] Epoch 3/50, Avg Loss=0.032007
[Siamese] Epoch 4/50, Avg Loss=0.031766
[Siamese] Epoch 5/50, Avg Loss=0.031560
[Siamese] Epoch 6/50, Avg Loss=0.031488
[Siamese] Epoch 7/50, Avg Loss=0.031382
[Siamese] Epoch 8/50, Avg Loss=0.031302
[Siamese] Epoch 9/50, Avg Loss=0.031186
[Siamese] Epoch 10/50, Avg Loss=0.031196
[Siamese] Epoch 11/50, Avg Loss=0.031111
[Siamese] Epoch 12/50, Avg Loss=0.031091
[Siamese] Epoch 13/50, Avg Loss=0.030980
[Siamese] Epoch 14/50, Avg Loss=0.030971
[Siamese] Epoch 15/50, Avg Loss=0.031023
[Siamese] Epoch 16/50, Avg Loss=0.030973
[Siamese] Epoch 17/50, Avg Loss=0.030961
[Siamese] Epoch 18/50, Avg Loss=0.030943
[Siamese] Epoch 19/50, Avg Loss=0.030868
[Siamese] Epoch 20/50, Avg Loss=0.030907
[Siamese] Epoch 21/50, Avg Loss=0.030875
[Siamese] Epoch 22/50, Avg Loss=0.030803
[Siamese] Epoch 23/50, Avg Loss=0.030831
[Siamese] Epoch 24/50, Avg Lo

Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Balanced subset: (3583, 768) (3583,)
[Siamese] Epoch 1/50, Avg Loss=0.034229
[Siamese] Epoch 2/50, Avg Loss=0.031532
[Siamese] Epoch 3/50, Avg Loss=0.031339
[Siamese] Epoch 4/50, Avg Loss=0.030673
[Siamese] Epoch 5/50, Avg Loss=0.030013
[Siamese] Epoch 6/50, Avg Loss=0.029796
[Siamese] Epoch 7/50, Avg Loss=0.029693
[Siamese] Epoch 8/50, Avg Loss=0.029564
[Siamese] Epoch 9/50, Avg Loss=0.029653
[Siamese] Epoch 10/50, Avg Loss=0.029666
[Siamese] Epoch 11/50, Avg Loss=0.029967
[Siamese] Epoch 12/50, Avg Loss=0.029426
[Siamese] Epoch 13/50, Avg Loss=0.029424
[Siamese] Epoch 14/50, Avg Loss=0.029337
[Siamese] Epoch 15/50, Avg Loss=0.029345
[Siamese] Epoch 16/50, Avg Loss=0.029355
[Siamese] Epoch 17/50, Avg Loss=0.029372
[Siamese] Epoch 18/50, Avg Loss=0.029362
[Siamese] Epoch 19/50, Avg Loss=0.029372
[Siamese] Epoch 20/50, Avg Loss=0.029374
[Siamese] Epoch 21/50, Avg Loss=0.029342
[Siamese] Epoch 22/50, Avg Loss=0.029289
[Siamese] Epoch 23/50, Avg Loss=0.029285
[Siamese] Epoch 24/50, Avg Lo

...

Balanced subset: (3583, 768) (3583,)
[Siamese] Epoch 1/50, Avg Loss=0.034566
[Siamese] Epoch 2/50, Avg Loss=0.031598
[Siamese] Epoch 3/50, Avg Loss=0.031337
[Siamese] Epoch 4/50, Avg Loss=0.031201
[Siamese] Epoch 5/50, Avg Loss=0.030980
[Siamese] Epoch 6/50, Avg Loss=0.030816
[Siamese] Epoch 7/50, Avg Loss=0.030794
[Siamese] Epoch 8/50, Avg Loss=0.030712
[Siamese] Epoch 9/50, Avg Loss=0.030671
[Siamese] Epoch 10/50, Avg Loss=0.030547
[Siamese] Epoch 11/50, Avg Loss=0.030381
[Siamese] Epoch 12/50, Avg Loss=0.030200
[Siamese] Epoch 13/50, Avg Loss=0.030249
[Siamese] Epoch 14/50, Avg Loss=0.030168
[Siamese] Epoch 15/50, Avg Loss=0.030112
[Siamese] Epoch 16/50, Avg Loss=0.030115
[Siamese] Epoch 17/50, Avg Loss=0.030103
[Siamese] Epoch 18/50, Avg Loss=0.030127
[Siamese] Epoch 19/50, Avg Loss=0.030037
[Siamese] Epoch 20/50, Avg Loss=0.030149
[Siamese] Epoch 21/50, Avg Loss=0.030011
[Siamese] Epoch 22/50, Avg Loss=0.029924
[Siamese] Epoch 23/50, Avg Loss=0.029964
[Siamese] Epoch 24/50, Avg Lo

Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Balanced subset: (3583, 768) (3583,)
[Siamese] Epoch 1/50, Avg Loss=0.034726
[Siamese] Epoch 2/50, Avg Loss=0.030826
[Siamese] Epoch 3/50, Avg Loss=0.030330
[Siamese] Epoch 4/50, Avg Loss=0.030113
[Siamese] Epoch 5/50, Avg Loss=0.029910
[Siamese] Epoch 6/50, Avg Loss=0.029805
[Siamese] Epoch 7/50, Avg Loss=0.029796
[Siamese] Epoch 8/50, Avg Loss=0.029659
[Siamese] Epoch 9/50, Avg Loss=0.029606
[Siamese] Epoch 10/50, Avg Loss=0.029652
[Siamese] Epoch 11/50, Avg Loss=0.029497
[Siamese] Epoch 12/50, Avg Loss=0.029585
[Siamese] Epoch 13/50, Avg Loss=0.029502
[Siamese] Epoch 14/50, Avg Loss=0.029629
[Siamese] Epoch 15/50, Avg Loss=0.029433
[Siamese] Epoch 16/50, Avg Loss=0.029455
[Siamese] Epoch 17/50, Avg Loss=0.029360
[Siamese] Epoch 18/50, Avg Loss=0.029386
[Siamese] Epoch 19/50, Avg Loss=0.029309
[Siamese] Epoch 20/50, Avg Loss=0.029403
[Siamese] Epoch 21/50, Avg Loss=0.029295
[Siamese] Epoch 22/50, Avg Loss=0.030117
[Siamese] Epoch 23/50, Avg Loss=0.030712
[Siamese] Epoch 24/50, Avg Lo

Using cache found in /home/snu/.cache/torch/hub/facebookresearch_dino_main


Balanced subset: (3583, 768) (3583,)
[Siamese] Epoch 1/50, Avg Loss=0.034495
[Siamese] Epoch 2/50, Avg Loss=0.032113
[Siamese] Epoch 3/50, Avg Loss=0.031901
[Siamese] Epoch 4/50, Avg Loss=0.031676
[Siamese] Epoch 5/50, Avg Loss=0.031480
[Siamese] Epoch 6/50, Avg Loss=0.031411
[Siamese] Epoch 7/50, Avg Loss=0.031311
[Siamese] Epoch 8/50, Avg Loss=0.031208
[Siamese] Epoch 9/50, Avg Loss=0.031226
[Siamese] Epoch 10/50, Avg Loss=0.031105
[Siamese] Epoch 11/50, Avg Loss=0.031095
[Siamese] Epoch 12/50, Avg Loss=0.031050
[Siamese] Epoch 13/50, Avg Loss=0.030994
[Siamese] Epoch 14/50, Avg Loss=0.030874
[Siamese] Epoch 15/50, Avg Loss=0.030917
[Siamese] Epoch 16/50, Avg Loss=0.030921
[Siamese] Epoch 17/50, Avg Loss=0.030954
[Siamese] Epoch 18/50, Avg Loss=0.030846
[Siamese] Epoch 19/50, Avg Loss=0.030856
[Siamese] Epoch 20/50, Avg Loss=0.030830
[Siamese] Epoch 21/50, Avg Loss=0.030844
[Siamese] Epoch 22/50, Avg Loss=0.030874
[Siamese] Epoch 23/50, Avg Loss=0.030800
[Siamese] Epoch 24/50, Avg Lo

=== FINAL SUMMARY ===
  accuracy | mean=0.9032 ± 0.0126
 precision | mean=0.4878 ± 0.4007
    recall | mean=0.4239 ± 0.3377
        f1 | mean=0.4533 ± 0.3663
  log_loss | mean=0.6941 ± 0.0080